# Online word2vec tutorial

So far, word2vec cannot increase the size of vocabulary after initial training. To handle unknown words, not in word2vec vocaburary, you must  retrain updated documents over again.

In this tutorial, we introduce gensim new feature, online vocaburary update. This additional feature overcomes the unknown word problems. Despite after initial training, we can continuously add new vocaburary to the pre-trained word2vec model using this online feature.

This implementation is still beta version at 16/09/04. You can download the beta version of online word2vec implementation in the following repository.

In [ ]:
%%bash
git clone -b online-w2v git@github.com:isohyt/gensim.git

In [1]:
from gensim.corpora.wikicorpus import WikiCorpus
from gensim.models.word2vec import Word2Vec, LineSentence
from pprint import pprint
from copy import deepcopy
from multiprocessing import cpu_count

### Download wikipedia dump files

We use the past and the current version of wiki dump files as online training.

In [ ]:
%%bash
wget https://dumps.wikimedia.org/archive/2010/2010-11/enwiki/20101011/enwiki-20101011-pages-articles.xml.bz2
wget https://dumps.wikimedia.org/enwiki/20160820/enwiki-20160820-pages-articles.xml.bz2

### Convert two wikipedia dump files
To avoid alert when convert old verision of wikipedia dump, you should download alternative wikicorpus.py in my repo.

In [2]:
old, new = [WikiCorpus('enwiki-{}-pages-articles.xml.bz2'.format(ymd)) for ymd in ['20101011', '20160820']]

In [3]:
def write_wiki(wiki):
    with open('{}.wiki'.format(wiki), 'w') as f:
        for text in eval(wiki).get_texts():
            f.write(' '.join([t.decode("utf-8") for t in text]) + "\n")

In [4]:
for wiki in ['old', 'new']:
    write_wiki(wiki)

In [5]:
oldwiki, newwiki = [LineSentence(f+'.wiki') for f in ['old', 'new']]

### Initial training
At first we train word2vec using "enwiki-20101011-pages-articles.xml.bz2". After that, we update model using "enwiki-20160820-pages-articles.xml.bz2".

In [6]:
model = Word2Vec(oldwiki, min_count = 0, workers=cpu_count())
# model = Word2Vec.load('oldmodel')
oldmodel = deepcopy(model)
oldmodel.save('oldmodel')

#### Japanese new idol group, ["Babymetal"](https://en.wikipedia.org/wiki/Babymetal), weren't known worldwide in 2010, so that the word, "babymetal", is not in oldmodel vocaburary.
Note: In recent years, they became the famous idol group not only in Japan. They won many music awards and run world tour.

In [7]:
try:
    print(oldmodel.most_similar('babymetal'))
except KeyError as e:
    print(e)

"word 'babymetal' not in vocabulary"


## Online update
To use online word2vec feature, set update=True when you use build_vocab using new documents.

In [8]:
model.build_vocab(newwiki, update=True)
model.train(newwiki)
model.save('newmodel')
# model = Word2Vec.load('newmodel')

#### Model Comparison
By the online training, the size of vocaburaries are increased about 3 millions.

In [9]:
for m in ['oldmodel', 'model']:
    print('The vocabulary size of the', m, 'is', len(eval(m).vocab))

The vocabulary size of the oldmodel is 6161170
The vocabulary size of the model is 9577157


#### After online training, the word, "babymetal", is added in model. This word is simillar with rock and metal bands.

In [10]:
try:
    pprint(model.most_similar('babymetal'))
except KeyError as e:
    print(e)

[('espairsray', 0.8282548189163208),
 ('mucc', 0.8032759428024292),
 ('galneryus', 0.7984307408332825),
 ('quruli', 0.7982708215713501),
 ('ellegarden', 0.7801949381828308),
 ('crossfaith', 0.779990017414093),
 ('chatmonchy', 0.7631145119667053),
 ('kuroyume', 0.7630795836448669),
 ('boøwy', 0.7613781690597534),
 ('ayabie', 0.7611944675445557)]


## The word, "Zootopia", become disney movie through the years.
In the past, the word, "Zootopia", was used just for an annual summer concert put on by New York top-40 radio station Z100, so that the word, "zootopia", is simillar with music festival.

In 2016, Zootopia is a American 3D computer-animated comedy film released by Walt Disney Pictures. As a result, the word, "zootopia", was often used as Animation films.

In [11]:
w = 'zootopia'
for m in ['oldmodel', 'model']:
    print('The count of the word,'+w+', is', eval(m).vocab[w].count, 'in', m)
    pprint(eval(m).most_similar(w))
    print('')

The count of the word,zootopia, is 24 in oldmodel
[('ramfest', 0.6304245591163635),
 ('concentratd', 0.6224538683891296),
 ('panagaris', 0.6218596696853638),
 ('fswe', 0.6206151843070984),
 ('soulico', 0.6189069747924805),
 ('tipitinia', 0.6183321475982666),
 ('plavka', 0.6179454326629639),
 ('rieleymike', 0.6165690422058105),
 ('selimova', 0.6146136522293091),
 ('silvashado', 0.6124839782714844)]

The count of the word,zootopia, is 502 in model
[('paranorman', 0.7775266170501709),
 ('antz', 0.7768376469612122),
 ('boxtrolls', 0.7680517435073853),
 ('croods', 0.76442950963974),
 ('incredibles', 0.7581683993339539),
 ('frankenweenie', 0.7470959424972534),
 ('pagemaster', 0.7417251467704773),
 ('shrek', 0.7348070740699768),
 ('megamind', 0.7298991680145264),
 ('spaceballs', 0.7187278270721436)]

